In [13]:
%AddJar https://oss.sonatype.org/content/repositories/snapshots/se/uu/it/cp/0.0.1-SNAPSHOT/cp-0.0.1-20170513.155506-6.jar

Using cached version of cp-0.0.1-20170513.155506-6.jar


In [18]:
{
    
import scala.io.Source
    
import se.uu.it.cp.UnderlyingAlgorithm

// Parse dataset
case class IrisData(
    sepalLenght: Double, 
    sepalWidth: Double, 
    petalLength: Double, 
    petalWidth: Double,
    species: String
)
val dataset = Source.fromFile("data/iris.data").getLines
    .map{ line =>
        val split = line.split(",")
        IrisData(split(0).toDouble, split(1).toDouble, 
            split(2).toDouble, split(3).toDouble, split(4))
    }
    .toSeq
    
// Split dataset
val (training,test) = dataset.splitAt(100)
val (properTraining, calibration) = dataset.splitAt(80)
    
// Define 5NN underlying algorithm
val alg = new UnderlyingAlgorithm(properTraining) {
    // Define how to make an IrisData from a feature sequence and a label
    def makeDataPoint(features: Seq[Double], label: Double) = {
        val species = label match {
            case 0.0 => "Iris-setosa"
            case 1.0 => "Iris-versicolor"
            case 2.0 => "Iris-virginica"
        }
        IrisData(features(0), features(1), features(2), 
                 features(3), species)
    }
    // Define how to get feature sequence from IrisData
    def getDataPointFeatures(dataPoint: IrisData) = {
        Seq(dataPoint.sepalLenght, dataPoint.sepalWidth, 
            dataPoint.petalLenght, dataPoint.petalWidth)
    }
    // Define how to get Double representation of the label from IrisData
    def getDataPointLabel(dataPoint: IrisData) = {
        dataPoint.species match {
            case "Iris-setosa" => 0.0
            case "Iris-versicolor" => 1.0
            case "Iris-virginica" => 2.0
        }
    }
    // Define training procedure
    def trainingProcedure(trainingSet: Seq[IrisData]) = (features: Seq[Double]) => {
      val trainByDistance = trainingSet.sortBy { point =>
        val squaredDist = getDataPointFeatures(point).zip(features).map {
          case (q, p) => math.pow(q - p, 2)
        }.reduce(_ + _)
        math.sqrt(squaredDist)
      }
      trainByDistance.take(5)
        .groupBy(getDataPointLabel(_))
        .maxBy(_._2.size)
        ._1
    }
    def nonConformityMeasure(newSample: IrisData) = {
      // Fraction of different closest neighbours
      val trainByDistance = this.properTrainingSet.sortBy { point =>
        val squaredDist = getDataPointFeatures(point)
        .zip(getDataPointFeatures(newSample)).map {
          case (q, p) => math.pow(q - p, 2)
        }.reduce(_ + _)
        math.sqrt(squaredDist)
      }
      val nOfUnconformal = trainByDistance.take(5)
        .count(getDataPointLabel(_) != getDataPointLabel(newSample))
      nOfUnconformal.toDouble / 5
    }
}
    
}

Name: Compile Error
Message: <console>:65: error: value petalLenght is not a member of IrisData
                   dataPoint.petalLenght, dataPoint.petalWidth)
                             ^
<console>:90: error: value properTrainingSet is not a member of se.uu.it.cp.UnderlyingAlgorithm[IrisData]
             val trainByDistance = this.properTrainingSet.sortBy { point =>
                                        ^
StackTrace: 